In [222]:
import pandas as pd 
import pip._vendor.requests as requests
import spotipy
import numpy as np
import matplotlib as plt
from spotipy.oauth2 import SpotifyOAuth
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.datasets import load_iris
from sklearn.multiclass import OutputCodeClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression 
from  sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
import pickle
scope1 = "user-library-read"
from spotipy.oauth2 import SpotifyClientCredentials
sp =spotipy.Spotify(auth_manager= SpotifyOAuth(client_id = "15fd015d4da742d0bf7da9c6ed3120c7", client_secret = "349c5181636e4e3cbb974b051db425fd", redirect_uri = "http://localhost", scope= scope1))

Happy song analysis

In [223]:
# df_happy1 = pd.read_csv("happy_songs_everyone_knows__.csv")
# df_happy2 = pd.read_csv("jhjh.csv")
# df_happy_total = pd.concat([df_happy1, df_happy2])
# df_happy_total = df_happy_total.drop(["Spotify ID", "Artist IDs", "Track Name", "Album Name", "Artist Name(s)", "Release Date"], axis = 1)
# df_happy_total["tag"] = "happy"
# df_happy_total.describe()
# # df_happy1[["Danceability","Energy","Speechiness","Acousticness", "Instrumentalness", "Liveness","Valence"]].plot.box(figsize = (10, 10))
# # # a = df_happy1.plot.scatter(x = "Popularity", y= "Energy")
# # # a
# # boxplot = df_happy1.boxplot(column =["Popularity","Danceability","Energy","Speechiness","Acousticness","Instrumentalness", "Liveness"])
# # boxplot
# # plt.show()

In [224]:
df_total = pd.read_csv("total.csv")

In [225]:
# df_total = df_total[~df_total['Tempo'].isin(df_vibe['Tempo'])]

In [226]:
df_total.loc[df_total["tag"] == "calm", "tag"] = 0
df_total.loc[df_total["tag"] == "happy", "tag"] = 1
df_total.loc[df_total["tag"] == "energ", "tag"] = 2
df_total.loc[df_total["tag"] == "sad", "tag"] = 3

In [227]:
df_total_after = df_total
df_total

,Danceability,Energy,Key,Loudness,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,tag
0,0.746,0.754,1,-6.684,0.0676,0.01520,0.000000,0.0486,0.746,149.882,2
1,0.628,0.593,8,-5.036,0.0323,0.09300,0.000002,0.0960,0.104,125.026,2
2,0.620,0.805,5,-5.538,0.0417,0.00564,0.000092,0.2850,0.586,122.062,2
3,0.504,0.785,8,-4.802,0.1040,0.07380,0.000000,0.2390,0.502,173.968,2
4,0.560,0.872,3,-4.269,0.1400,0.02390,0.000007,0.3710,0.682,126.050,2
...,...,...,...,...,...,...,...,...,...,...,...
1496,0.688,0.662,6,-5.570,0.0441,0.09710,0.000001,0.1720,0.549,115.106,1
1497,0.521,0.877,7,-5.444,0.0683,0.01320,0.080600,0.0929,0.199,180.007,1
1498,0.287,0.473,5,-3.551,0.0340,0.08580,0.000044,0.3830,0.577,186.283,1
1499,0.673,0.704,6,-8.056,0.3600,0.19600,0.000000,0.0898,0.372,86.919,1


In [228]:

# Normalize the DataFrame to be between 0 and 1
min_value = df_total[["Tempo"]].min()
max_value = df_total[["Tempo"]].max()
df_total["Tempo"] = (df_total[["Tempo"]] - min_value) / (max_value - min_value)

min_value = df_total[["Loudness"]].min()
max_value = df_total[["Loudness"]].max()
df_total["Loudness"] = (df_total[["Loudness"]] - min_value) / (max_value - min_value)

min_value = df_total[["Key"]].min()
max_value = df_total[["Key"]].max()
df_total["Key"] = (df_total[["Key"]] - min_value) / (max_value - min_value)



In [229]:
# Normalize the DataFrame to be between 0 and 1 function 
def clean_and_normalize(inpput):
    inpput = inpput[["Danceability", "Energy", "Key", "Loudness", "Speechiness", "Acousticness", "Instrumentalness", "Liveness", "Valence", "Tempo"]]
    min_value = inpput[["Tempo"]].min()
    max_value = inpput[["Tempo"]].max()
    inpput["Tempo"] = (inpput[["Tempo"]] - min_value) / (max_value - min_value)

    min_value = inpput[["Loudness"]].min()
    max_value = inpput[["Loudness"]].max()
    inpput["Loudness"] = (inpput[["Loudness"]] - min_value) / (max_value - min_value)

    min_value = inpput[["Key"]].min()
    max_value = inpput[["Key"]].max()
    inpput["Key"] = (inpput[["Key"]] - min_value) / (max_value - min_value)
    inpput



In [230]:
data = df_total[["Danceability", "Energy", "Key", "Loudness", "Speechiness", "Acousticness", "Instrumentalness", "Liveness", "Valence", "Tempo"]]

In [231]:
data_target = df_total[["tag"]].astype(int)
data_target.dtypes

tag    int64
dtype: object

In [232]:
# song_x_train, song_x_test, song_y_train, song_y_test = train_test_split(data, data_target, test_size= 0.2, random_state= 42)
# df_result = []

# for i in range(10):
#     glp = GradientBoostingClassifier(n_estimators=170, learning_rate=0.001, max_depth=5, min_samples_split=2, min_samples_leaf=1, random_state=42)
#     song_y_train = np.ravel(song_y_train)
#     song_y_test = np.ravel(song_y_test)
#     glp.fit(song_x_train,song_y_train)
#     song_y_predict = glp.predict(song_x_test)
#     output = {"RMSE": np.sqrt(mean_squared_error(song_y_test,song_y_predict)), "accuracy_score":metrics.accuracy_score(song_y_test, song_y_predict)}
#     df_result.append(output)
#         # print("RMSE", np.sqrt(mean_squared_error(song_y_test,song_y_predict)))
#         # print(metrics.accuracy_score(song_y_test, song_y_predict))
#         # print("------")
# df_output = pd.DataFrame(df_result)
# df_output

In [233]:
song_x_test

,Danceability,Energy,Key,Loudness,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo
1117,0.766,0.2830,0.727273,0.775361,0.0846,0.1110,0.017200,0.1150,0.144,0.326002
643,0.330,0.0109,0.545455,0.303344,0.0356,0.9820,0.924000,0.1120,0.208,0.237233
422,0.366,0.0329,0.909091,0.380450,0.0329,0.9940,0.909000,0.1160,0.211,0.340815
413,0.610,0.5080,0.727273,0.861144,0.1520,0.2970,0.000000,0.3840,0.758,0.822876
451,0.536,0.0409,0.000000,0.294008,0.0542,0.9920,0.946000,0.1070,0.282,0.182605
...,...,...,...,...,...,...,...,...,...,...
610,0.290,0.0197,0.454545,0.443329,0.0443,0.9940,0.903000,0.1110,0.319,0.090118
1343,0.730,0.6990,0.909091,0.845859,0.0434,0.0401,0.000003,0.1290,0.624,0.483537
654,0.495,0.0495,0.909091,0.332949,0.0358,0.9840,0.937000,0.1020,0.223,0.112586
619,0.436,0.0356,0.000000,0.453370,0.0409,0.9920,0.961000,0.0895,0.238,0.525850


In [234]:
song_x_train, song_x_test, song_y_train, song_y_test = train_test_split(data, data_target, test_size= 0.2, random_state= 42)
df_result = []


model = OneVsRestClassifier(MLPClassifier(hidden_layer_sizes= (16, 10), activation= "logistic",max_iter= 1000, learning_rate_init= 0.01, alpha = 0.01), n_jobs = -1)
song_y_train = np.ravel(song_y_train)
song_y_test = np.ravel(song_y_test)
model.fit(song_x_train,song_y_train)
df_total_after["prediciton"] = model.predict(df_total_after.drop("tag", axis= 1))
tag = np.ravel(df_total_after[["tag"]].astype(int))
prediction = np.ravel(df_total_after[["prediciton"]].astype(int))
output = {"RMSE": np.sqrt(mean_squared_error(tag,prediction)), "accuracy_score":metrics.accuracy_score(tag,prediction)}
# df_result.append(output)
#         # print("RMSE", np.sqrt(mean_squared_error(song_y_test,song_y_predict)))
#         # print(metrics.accuracy_score(song_y_test, song_y_predict))
#         # print("------")
# df_output = pd.DataFrame(df_result)
# df_output
df_total_after
# song_x_test

,Danceability,Energy,Key,Loudness,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,tag,prediciton
0,0.746,0.754,0.090909,0.861097,0.0676,0.01520,0.000000,0.0486,0.746,0.647064,2,2
1,0.628,0.593,0.727273,0.899850,0.0323,0.09300,0.000002,0.0960,0.104,0.496042,2,3
2,0.620,0.805,0.454545,0.888045,0.0417,0.00564,0.000092,0.2850,0.586,0.478033,2,2
3,0.504,0.785,0.727273,0.905352,0.1040,0.07380,0.000000,0.2390,0.502,0.793408,2,2
4,0.560,0.872,0.272727,0.917886,0.1400,0.02390,0.000007,0.3710,0.682,0.502263,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...
1496,0.688,0.662,0.545455,0.887292,0.0441,0.09710,0.000001,0.1720,0.549,0.435769,1,2
1497,0.521,0.877,0.636364,0.890255,0.0683,0.01320,0.080600,0.0929,0.199,0.830100,1,2
1498,0.287,0.473,0.454545,0.934769,0.0340,0.08580,0.000044,0.3830,0.577,0.868232,1,3
1499,0.673,0.704,0.545455,0.828834,0.3600,0.19600,0.000000,0.0898,0.372,0.264508,1,2


In [235]:
output

{'RMSE': 1.0082933913093093, 'accuracy_score': 0.6868754163890739}

In [236]:
df_difference = df_total_after[df_total_after.tag != df_total_after.prediciton]
df_difference[df_difference["tag"] == 2]

,Danceability,Energy,Key,Loudness,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,tag,prediciton
1,0.628,0.593,0.727273,0.899850,0.0323,0.093000,0.000002,0.0960,0.104,0.496042,2,3
6,0.527,0.835,0.545455,0.893689,0.0433,0.016600,0.000000,0.2490,0.654,0.501856,2,1
7,0.572,0.797,0.000000,0.919955,0.0497,0.094900,0.000000,0.2590,0.598,0.561260,2,1
10,0.691,0.602,0.090909,0.828670,0.0497,0.051500,0.000000,0.1410,0.715,0.458827,2,1
12,0.691,0.795,0.090909,0.923717,0.0355,0.000812,0.000001,0.1560,0.876,0.550390,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...
399,0.645,0.534,0.363636,0.890632,0.0306,0.147000,0.000004,0.0858,0.302,0.283373,2,3
403,0.639,0.698,0.636364,0.854277,0.0702,0.756000,0.000000,0.1460,0.310,0.442823,2,3
405,0.802,0.848,1.000000,0.871091,0.0465,0.011800,0.395000,0.0567,0.165,0.465273,2,3
406,0.545,0.473,1.000000,0.824061,0.0312,0.284000,0.000000,0.0871,0.195,0.526263,2,3


In [237]:
# mean = df_output["accuracy_score"].mean()
# std = df_output["accuracy_score"].std()
# normal = np.random.normal(mean, std, size = len(df_output))
# plt.hist(normal, bins = 50, density= True)

In [238]:
# mean = df_output["RMSE"].mean()
# std = df_output["RMSE"].std()
# normal = np.random.normal(mean, std, size = len(df_output))
# plt.hist(normal, bins = 30, density= True)

In [239]:
filename = "model.pkl"
with open (filename, "wb") as file:
    pickle.dump(model, file)

# load the saved model from file
filename = 'model.pkl'
with open(filename, 'rb') as file:
    loaded_model = pickle.load(file)


In [315]:

test_energy = pd.read_csv("engertic.csv")
test_energy

,Spotify ID,Artist IDs,Track Name,Album Name,Artist Name(s),Release Date,Duration (ms),Popularity,Added By,Added At,...,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Time Signature
0,3zkWCteF82vJwv0hRLba76,7n2wHs1TKAczGzO7Dd2rGr,Stitches,Handwritten,Shawn Mendes,2015-04-14,206880,79,spotify:user:,2023-01-27T11:00:00Z,...,1,-6.684,1,0.0676,0.01520,0.000000,0.0486,0.746,149.882,4
1,7xoUc6faLbCqZO6fQEYprd,66CXWjxzNUsdJxJ2JdwvnR,One Last Time,My Everything (Deluxe),Ariana Grande,2014-08-25,197266,82,spotify:user:,2023-01-27T11:00:00Z,...,8,-5.036,1,0.0323,0.09300,0.000002,0.0960,0.104,125.026,4
2,2hHLbkatPwOOmrNxTiD41L,6sFIWsNpZYqfjUpaCgueju,I Really Like You,Emotion (Deluxe Expanded Edition),Carly Rae Jepsen,2015,204763,69,spotify:user:,2023-01-27T11:00:00Z,...,5,-5.538,1,0.0417,0.00564,0.000092,0.2850,0.586,122.062,4
3,1V6gIisPpYqgFeWbMLI0bA,6S2OmqARrzebs0tKUEyXyp,Heart Attack,Demi,Demi Lovato,2013-01-01,210840,76,spotify:user:,2023-01-27T11:00:00Z,...,8,-4.802,1,0.1040,0.07380,0.000000,0.2390,0.502,173.968,4
4,1kPpge9JDLpcj15qgrPbYX,"07QEuhtrNmmZ0zEcqE9SF6,6sFIWsNpZYqfjUpaCgueju",Good Time,The Midsummer Station,"Owl City,Carly Rae Jepsen",2012-01-01,205933,76,spotify:user:,2023-01-27T11:00:00Z,...,3,-4.269,1,0.1400,0.02390,0.000007,0.3710,0.682,126.050,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,6BePGk3eCan4FqaW2X8Qy3,"2o5jDhtHVPhrJdv3cEQ99Z,45dkTj5sMRSjrmBSBeiHym",10:35,10:35,"Tiësto,Tate McRae",2022-11-03,172252,89,spotify:user:,2023-01-27T11:00:00Z,...,8,-5.733,1,0.0970,0.06830,0.000004,0.1800,0.698,120.003,4
146,0U10zFw4GlBacOy9VDGfGL,"5pKCCKE2ajJHZ9KAiaK11H,7CajNmpbOovFoOoasH2HaY",We Found Love,Talk That Talk,"Rihanna,Calvin Harris",2011-11-19,215226,80,spotify:user:,2023-01-27T11:00:00Z,...,1,-4.485,1,0.0383,0.02500,0.001380,0.1080,0.600,127.985,4
147,4zP7ADsgJgHGY6VzxbNp1z,7gOdHgIoIKoe4i9Tta6qdD,Year 3000,Music From Chasing Happiness,Jonas Brothers,2019-05-09,201960,63,spotify:user:,2023-01-27T11:00:00Z,...,11,-5.850,1,0.0437,0.00450,0.000002,0.3350,0.798,106.965,4
148,48QmG1dfvMuYLxMPt7KSRA,"3Isy6kedDrgPYoTS1dazA9,6M2wZ9GZgrQXHCFfjv46we",No Lie,Mad Love The Prequel,"Sean Paul,Dua Lipa",2018-06-29,221176,80,spotify:user:,2023-01-27T11:00:00Z,...,7,-2.862,1,0.1170,0.04660,0.000000,0.2060,0.463,102.040,4


In [ ]:
df = pd.DataFrame()
for i in range (len(test_energy)) :
    song_uri = test_energy.at[i, "Track URI"]
    analysis = sp.audio_features(song_uri)
    analy = pd.DataFrame(analysis)
    df = pd.concat([df, analy])
test_energy = df.reset_index().drop("index", axis= 1)

In [298]:
big = ["Danceability", "Energy", "Key", "Loudness", "Speechiness", "Acousticness", "Instrumentalness", "Liveness", "Valence", "Tempo"]
small = ["danceability", "energy", "key", "loudness", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo"]

In [319]:
# test_energy = test_energy[["danceability", "energy", "key", "loudness", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo"]]
# Normalize the DataFrame to be between 0 and 1
min_value = test_energy[["Tempo"]].min()
max_value = test_energy[["Tempo"]].max()
test_energy["Tempo"] = (test_energy[["Tempo"]] - min_value) / (max_value - min_value)

min_value = test_energy[["Loudness"]].min()
max_value = test_energy[["Loudness"]].max()
test_energy["Loudness"] = (test_energy[["Loudness"]] - min_value) / (max_value - min_value)

min_value = test_energy[["Key"]].min()
max_value = test_energy[["Key"]].max()
test_energy["Key"] = (test_energy[["Key"]] - min_value) / (max_value - min_value)
test_energy

,Spotify ID,Artist IDs,Track Name,Album Name,Artist Name(s),Release Date,Duration (ms),Popularity,Added By,Added At,...,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Time Signature
0,3zkWCteF82vJwv0hRLba76,7n2wHs1TKAczGzO7Dd2rGr,Stitches,Handwritten,Shawn Mendes,2015-04-14,206880,79,spotify:user:,2023-01-27T11:00:00Z,...,0.090909,0.247630,1,0.0676,0.01520,0.000000,0.0486,0.746,0.611228,4
1,7xoUc6faLbCqZO6fQEYprd,66CXWjxzNUsdJxJ2JdwvnR,One Last Time,My Everything (Deluxe),Ariana Grande,2014-08-25,197266,82,spotify:user:,2023-01-27T11:00:00Z,...,0.727273,0.521703,1,0.0323,0.09300,0.000002,0.0960,0.104,0.369560,4
2,2hHLbkatPwOOmrNxTiD41L,6sFIWsNpZYqfjUpaCgueju,I Really Like You,Emotion (Deluxe Expanded Edition),Carly Rae Jepsen,2015,204763,69,spotify:user:,2023-01-27T11:00:00Z,...,0.454545,0.438217,1,0.0417,0.00564,0.000092,0.2850,0.586,0.340742,4
3,1V6gIisPpYqgFeWbMLI0bA,6S2OmqARrzebs0tKUEyXyp,Heart Attack,Demi,Demi Lovato,2013-01-01,210840,76,spotify:user:,2023-01-27T11:00:00Z,...,0.727273,0.560619,1,0.1040,0.07380,0.000000,0.2390,0.502,0.845409,4
4,1kPpge9JDLpcj15qgrPbYX,"07QEuhtrNmmZ0zEcqE9SF6,6sFIWsNpZYqfjUpaCgueju",Good Time,The Midsummer Station,"Owl City,Carly Rae Jepsen",2012-01-01,205933,76,spotify:user:,2023-01-27T11:00:00Z,...,0.272727,0.649260,1,0.1400,0.02390,0.000007,0.3710,0.682,0.379516,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,6BePGk3eCan4FqaW2X8Qy3,"2o5jDhtHVPhrJdv3cEQ99Z,45dkTj5sMRSjrmBSBeiHym",10:35,10:35,"Tiësto,Tate McRae",2022-11-03,172252,89,spotify:user:,2023-01-27T11:00:00Z,...,0.727273,0.405787,1,0.0970,0.06830,0.000004,0.1800,0.698,0.320723,4
146,0U10zFw4GlBacOy9VDGfGL,"5pKCCKE2ajJHZ9KAiaK11H,7CajNmpbOovFoOoasH2HaY",We Found Love,Talk That Talk,"Rihanna,Calvin Harris",2011-11-19,215226,80,spotify:user:,2023-01-27T11:00:00Z,...,0.090909,0.613338,1,0.0383,0.02500,0.001380,0.1080,0.600,0.398330,4
147,4zP7ADsgJgHGY6VzxbNp1z,7gOdHgIoIKoe4i9Tta6qdD,Year 3000,Music From Chasing Happiness,Jonas Brothers,2019-05-09,201960,63,spotify:user:,2023-01-27T11:00:00Z,...,1.000000,0.386330,1,0.0437,0.00450,0.000002,0.3350,0.798,0.193958,4
148,48QmG1dfvMuYLxMPt7KSRA,"3Isy6kedDrgPYoTS1dazA9,6M2wZ9GZgrQXHCFfjv46we",No Lie,Mad Love The Prequel,"Sean Paul,Dua Lipa",2018-06-29,221176,80,spotify:user:,2023-01-27T11:00:00Z,...,0.636364,0.883253,1,0.1170,0.04660,0.000000,0.2060,0.463,0.146074,4


In [300]:
test_energy[["Danceability", "Energy", "Key", "Loudness", "Speechiness", "Acousticness", "Instrumentalness", "Liveness", "Valence", "Tempo"]] = test_energy[["danceability", "energy", "key", "loudness", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo"]] 

In [301]:
test_energy = test_energy.drop(["danceability", "energy", "key", "loudness", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo"], axis= 1)

In [320]:
test_energy["prediciton"] = model.predict(test_energy[["Danceability", "Energy", "Key", "Loudness", "Speechiness", "Acousticness", "Instrumentalness", "Liveness", "Valence", "Tempo"]] )

In [321]:
test_energy

,Spotify ID,Artist IDs,Track Name,Album Name,Artist Name(s),Release Date,Duration (ms),Popularity,Added By,Added At,...,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Time Signature,prediciton
0,3zkWCteF82vJwv0hRLba76,7n2wHs1TKAczGzO7Dd2rGr,Stitches,Handwritten,Shawn Mendes,2015-04-14,206880,79,spotify:user:,2023-01-27T11:00:00Z,...,0.247630,1,0.0676,0.01520,0.000000,0.0486,0.746,0.611228,4,2
1,7xoUc6faLbCqZO6fQEYprd,66CXWjxzNUsdJxJ2JdwvnR,One Last Time,My Everything (Deluxe),Ariana Grande,2014-08-25,197266,82,spotify:user:,2023-01-27T11:00:00Z,...,0.521703,1,0.0323,0.09300,0.000002,0.0960,0.104,0.369560,4,3
2,2hHLbkatPwOOmrNxTiD41L,6sFIWsNpZYqfjUpaCgueju,I Really Like You,Emotion (Deluxe Expanded Edition),Carly Rae Jepsen,2015,204763,69,spotify:user:,2023-01-27T11:00:00Z,...,0.438217,1,0.0417,0.00564,0.000092,0.2850,0.586,0.340742,4,2
3,1V6gIisPpYqgFeWbMLI0bA,6S2OmqARrzebs0tKUEyXyp,Heart Attack,Demi,Demi Lovato,2013-01-01,210840,76,spotify:user:,2023-01-27T11:00:00Z,...,0.560619,1,0.1040,0.07380,0.000000,0.2390,0.502,0.845409,4,3
4,1kPpge9JDLpcj15qgrPbYX,"07QEuhtrNmmZ0zEcqE9SF6,6sFIWsNpZYqfjUpaCgueju",Good Time,The Midsummer Station,"Owl City,Carly Rae Jepsen",2012-01-01,205933,76,spotify:user:,2023-01-27T11:00:00Z,...,0.649260,1,0.1400,0.02390,0.000007,0.3710,0.682,0.379516,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,6BePGk3eCan4FqaW2X8Qy3,"2o5jDhtHVPhrJdv3cEQ99Z,45dkTj5sMRSjrmBSBeiHym",10:35,10:35,"Tiësto,Tate McRae",2022-11-03,172252,89,spotify:user:,2023-01-27T11:00:00Z,...,0.405787,1,0.0970,0.06830,0.000004,0.1800,0.698,0.320723,4,2
146,0U10zFw4GlBacOy9VDGfGL,"5pKCCKE2ajJHZ9KAiaK11H,7CajNmpbOovFoOoasH2HaY",We Found Love,Talk That Talk,"Rihanna,Calvin Harris",2011-11-19,215226,80,spotify:user:,2023-01-27T11:00:00Z,...,0.613338,1,0.0383,0.02500,0.001380,0.1080,0.600,0.398330,4,2
147,4zP7ADsgJgHGY6VzxbNp1z,7gOdHgIoIKoe4i9Tta6qdD,Year 3000,Music From Chasing Happiness,Jonas Brothers,2019-05-09,201960,63,spotify:user:,2023-01-27T11:00:00Z,...,0.386330,1,0.0437,0.00450,0.000002,0.3350,0.798,0.193958,4,2
148,48QmG1dfvMuYLxMPt7KSRA,"3Isy6kedDrgPYoTS1dazA9,6M2wZ9GZgrQXHCFfjv46we",No Lie,Mad Love The Prequel,"Sean Paul,Dua Lipa",2018-06-29,221176,80,spotify:user:,2023-01-27T11:00:00Z,...,0.883253,1,0.1170,0.04660,0.000000,0.2060,0.463,0.146074,4,2


In [322]:
calm = test_energy[test_energy['prediciton'] == 0 ]
happy = test_energy[test_energy['prediciton'] == 1 ]
energy = test_energy[test_energy['prediciton'] == 2 ]
sad = test_energy[test_energy['prediciton'] == 3 ]

In [323]:
print(len(calm))
calm

0


,Spotify ID,Artist IDs,Track Name,Album Name,Artist Name(s),Release Date,Duration (ms),Popularity,Added By,Added At,...,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Time Signature,prediciton


In [324]:
print(len(happy))
happy

19


,Spotify ID,Artist IDs,Track Name,Album Name,Artist Name(s),Release Date,Duration (ms),Popularity,Added By,Added At,...,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Time Signature,prediciton
6,0ct6r3EGTcMLPtrXHDvVjc,1vCWHaC5f2uS3yhpwWbIA6,The Nights,The Days / Nights,Avicii,2014-01-01,176658,84,spotify:user:,2023-01-27T11:00:00Z,...,0.478131,1,0.0433,0.016600,0.000000,0.2490,0.654,0.378865,4,1
7,07J3p1GchLPc2kAKFgd1TD,3zDRCqOhJXJfS2YWOEwGMC,Fall So Hard,Fall So Hard,Christopher,2021-03-12,214720,53,spotify:user:,2023-01-27T11:00:00Z,...,0.663895,1,0.0497,0.094900,0.000000,0.2590,0.598,0.473924,4,1
12,4uQY80TKE1u4ZQZQKHzVEi,6jJ0s89eD6GaHleKKya26X,The One That Got Away,Teenage Dream,Katy Perry,2010-08-24,227333,76,spotify:user:,2023-01-27T11:00:00Z,...,0.690504,0,0.0355,0.000812,0.000001,0.1560,0.876,0.456530,4,1
24,7oUHzZKNKUTrlU7DYxhQ7v,04gDigrS5kc9YWfZHwBETP,Maps,V (Asia Tour Edition),Maroon 5,2015-05-18,189960,71,spotify:user:,2023-01-27T11:00:00Z,...,0.440878,0,0.0303,0.020500,0.000000,0.0590,0.879,0.321005,4,1
27,6Vh03bkEfXqekWp7Y1UBRb,4AK6F7OLvEQ5QYCBNiQWHq,Live While We're Young,Take Me Home (Expanded Edition),One Direction,2012-11-12,200213,73,spotify:user:,2023-01-27T11:00:00Z,...,1.000000,1,0.0544,0.054200,0.000000,0.1440,0.931,0.379409,4,1
29,0hO71r8vbnrXSaXVrR9ptF,1l8Fu6IkuTP0U5QetQJ5Xt,I'm In Love With a Monster,I'm In Love With a Monster,Fifth Harmony,2015-08-14,211932,64,spotify:user:,2023-01-27T11:00:00Z,...,0.682355,0,0.0647,0.059700,0.000000,0.0603,0.868,0.564335,4,1
42,42loEE51UDcecom9K8K4ei,6aZyMrc4doVtZyKNilOmwu,I Do,All Of You,Colbie Caillat,2011-01-01,173360,57,spotify:user:,2023-01-27T11:00:00Z,...,0.418593,1,0.0329,0.220000,0.000000,0.0785,0.958,0.562566,4,1
64,3OOCTo40A47WiXqMu6NDi2,"1ZrBGJWLL8NiAjgNifCy90,599aGczoZFptUe5tbS43LT",Forgive Me Friend (feat. Swedish Jam Factory),Pixie's Parasol,"Smith & Thell,Swedish Jam Factory",2021-02-05,216133,56,spotify:user:,2023-01-27T11:00:00Z,...,0.402794,1,0.0319,0.001400,0.000000,0.1980,0.624,0.397659,4,1
69,76hfruVvmfQbw0eYn1nmeC,6T5tfhQCknKG4UnH90qGnz,Cake By The Ocean,DNCE,DNCE,2016-11-18,219146,79,spotify:user:,2023-01-27T11:00:00Z,...,0.453517,0,0.0517,0.152000,0.000000,0.0371,0.896,0.310991,4,1
72,7EQGXaVSyEDsCWKmUcfpLk,6LqNN22kT3074XbTVUrhzX,Die Young,Warrior (Expanded Edition),Kesha,2012-11-30,211920,76,spotify:user:,2023-01-27T11:00:00Z,...,0.560120,0,0.0460,0.004980,0.000125,0.2150,0.801,0.398485,4,1


In [325]:
print(len(energy))
energy

95


,Spotify ID,Artist IDs,Track Name,Album Name,Artist Name(s),Release Date,Duration (ms),Popularity,Added By,Added At,...,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Time Signature,prediciton
0,3zkWCteF82vJwv0hRLba76,7n2wHs1TKAczGzO7Dd2rGr,Stitches,Handwritten,Shawn Mendes,2015-04-14,206880,79,spotify:user:,2023-01-27T11:00:00Z,...,0.247630,1,0.0676,0.01520,0.000000,0.0486,0.746,0.611228,4,2
2,2hHLbkatPwOOmrNxTiD41L,6sFIWsNpZYqfjUpaCgueju,I Really Like You,Emotion (Deluxe Expanded Edition),Carly Rae Jepsen,2015,204763,69,spotify:user:,2023-01-27T11:00:00Z,...,0.438217,1,0.0417,0.00564,0.000092,0.2850,0.586,0.340742,4,2
4,1kPpge9JDLpcj15qgrPbYX,"07QEuhtrNmmZ0zEcqE9SF6,6sFIWsNpZYqfjUpaCgueju",Good Time,The Midsummer Station,"Owl City,Carly Rae Jepsen",2012-01-01,205933,76,spotify:user:,2023-01-27T11:00:00Z,...,0.649260,1,0.1400,0.02390,0.000007,0.3710,0.682,0.379516,4,2
8,7JJmb5XwzOO8jgpou264Ml,7n2wHs1TKAczGzO7Dd2rGr,There's Nothing Holdin' Me Back,Illuminate (Deluxe),Shawn Mendes,2017-04-20,199440,85,spotify:user:,2023-01-27T11:00:00Z,...,0.683519,0,0.0554,0.38000,0.000000,0.0779,0.969,0.340120,4,2
11,6rmXhRIemCTPyMYZRDN7Qg,3e7awlrlDSwF3iM0WBjGMp,Black Magic,Get Weird (Expanded Edition),Little Mix,2015-11-06,211786,73,spotify:user:,2023-01-27T11:00:00Z,...,0.438716,1,0.0575,0.03430,0.000000,0.3140,0.849,0.242795,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,6BePGk3eCan4FqaW2X8Qy3,"2o5jDhtHVPhrJdv3cEQ99Z,45dkTj5sMRSjrmBSBeiHym",10:35,10:35,"Tiësto,Tate McRae",2022-11-03,172252,89,spotify:user:,2023-01-27T11:00:00Z,...,0.405787,1,0.0970,0.06830,0.000004,0.1800,0.698,0.320723,4,2
146,0U10zFw4GlBacOy9VDGfGL,"5pKCCKE2ajJHZ9KAiaK11H,7CajNmpbOovFoOoasH2HaY",We Found Love,Talk That Talk,"Rihanna,Calvin Harris",2011-11-19,215226,80,spotify:user:,2023-01-27T11:00:00Z,...,0.613338,1,0.0383,0.02500,0.001380,0.1080,0.600,0.398330,4,2
147,4zP7ADsgJgHGY6VzxbNp1z,7gOdHgIoIKoe4i9Tta6qdD,Year 3000,Music From Chasing Happiness,Jonas Brothers,2019-05-09,201960,63,spotify:user:,2023-01-27T11:00:00Z,...,0.386330,1,0.0437,0.00450,0.000002,0.3350,0.798,0.193958,4,2
148,48QmG1dfvMuYLxMPt7KSRA,"3Isy6kedDrgPYoTS1dazA9,6M2wZ9GZgrQXHCFfjv46we",No Lie,Mad Love The Prequel,"Sean Paul,Dua Lipa",2018-06-29,221176,80,spotify:user:,2023-01-27T11:00:00Z,...,0.883253,1,0.1170,0.04660,0.000000,0.2060,0.463,0.146074,4,2


In [326]:
print(len(sad))
sad

36


,Spotify ID,Artist IDs,Track Name,Album Name,Artist Name(s),Release Date,Duration (ms),Popularity,Added By,Added At,...,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Time Signature,prediciton
1,7xoUc6faLbCqZO6fQEYprd,66CXWjxzNUsdJxJ2JdwvnR,One Last Time,My Everything (Deluxe),Ariana Grande,2014-08-25,197266,82,spotify:user:,2023-01-27T11:00:00Z,...,0.521703,1,0.0323,0.093000,0.000002,0.0960,0.104,0.369560,4,3
3,1V6gIisPpYqgFeWbMLI0bA,6S2OmqARrzebs0tKUEyXyp,Heart Attack,Demi,Demi Lovato,2013-01-01,210840,76,spotify:user:,2023-01-27T11:00:00Z,...,0.560619,1,0.1040,0.073800,0.000000,0.2390,0.502,0.845409,4,3
5,1yvMUkIOTeUNtNWlWRgANS,5WUlDfRSoLAfcVSX1WnrxN,Unstoppable,This Is Acting (Deluxe Version),Sia,2016-10-21,217746,81,spotify:user:,2023-01-27T11:00:00Z,...,0.554465,1,0.0779,0.112000,0.000373,0.1010,0.260,0.843766,4,3
9,6PCUP3dWmTjcTtXY02oFdT,6eUKZXaKkcviH0Ku9w2n3V,Castle on the Hill,÷ (Deluxe),Ed Sheeran,2017-03-03,261153,78,spotify:user:,2023-01-27T11:00:00Z,...,0.549642,1,0.0989,0.023200,0.000011,0.1400,0.471,0.466602,4,3
10,5gzvXEGf0XDseXgIlcSSLg,"2kUBwtoPkA9ZoJxcQUtL2P,05OqdicfTgKQVfyEOXfKRa",Bodies,Bodies,"CLAUDIA,Irwinandfire",2020-10-30,207731,47,spotify:user:,2023-01-27T11:00:00Z,...,0.018294,1,0.0497,0.051500,0.000000,0.1410,0.715,0.310009,4,3
14,5HCyWlXZPP0y6Gqq8TgA20,"2tIP7SsRs7vjIcLrU85W8J,1uNFoZAHBGtllmzznpCI3s",STAY (with Justin Bieber),STAY (with Justin Bieber),"The Kid LAROI,Justin Bieber",2021-07-09,141805,85,spotify:user:,2023-01-27T11:00:00Z,...,0.447198,1,0.0483,0.038300,0.000000,0.1030,0.478,0.806129,4,3
16,2bT1PH7Cw3J9p3t7nlXCdh,7n2wHs1TKAczGzO7Dd2rGr,If I Can't Have You,Shawn Mendes (Deluxe),Shawn Mendes,2019-06-19,191466,72,spotify:user:,2023-01-27T11:00:00Z,...,0.661234,1,0.0623,0.487000,0.000000,0.1340,0.870,0.358953,4,3
17,0FDzzruyVECATHXKHFs9eJ,4gzpq5DPGxSnKTe4SA8HAU,A Sky Full of Stars,Ghost Stories,Coldplay,2014-05-19,267866,82,spotify:user:,2023-01-27T11:00:00Z,...,0.282554,1,0.0279,0.006170,0.001970,0.2090,0.162,0.369016,4,3
22,22PMfvdz35fFKYnJyMn077,0du5cEVh5yTK9QJze8zA0C,Marry You,Doo-Wops & Hooligans,Bruno Mars,2010-10-05,230192,74,spotify:user:,2023-01-27T11:00:00Z,...,0.550141,1,0.0367,0.332000,0.000000,0.1040,0.452,0.562838,4,3
33,0nrRP2bk19rLc0orkWPQk2,1vCWHaC5f2uS3yhpwWbIA6,Wake Me Up,True,Avicii,2013-01-01,247426,84,spotify:user:,2023-01-27T11:00:00Z,...,0.411774,1,0.0523,0.003800,0.001200,0.1610,0.643,0.360362,4,3
